# Notebook for Loading in the Database and Creating a Graph

Start by importing the libraries and connecting to the neo4j sandbox.

In [ ]:
# imports
import pandas as pd
from graphdatascience import GraphDataScience

In [ ]:
# Connect to neo4j DBMS

#this is local DBMS via neo4j Desktop for testing purposes needs to be changed
DB_ULR = "bolt://localhost:7687" 
DB_USER = "neo4j"
DB_PASS = "1234"
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))

## Loading data into the graph

Preprocessed data stored in https://github.com/EY-Tech-Consulting-Denmark/Graphathon-ATP/tree/main/Data/clean_data can be loaded into a Neo4j sandbox.

### Loading the preprocessed data

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/EY-Tech-Consulting-Denmark/Graphathon-ATP/main/Data/clean_data/data.csv")
data.head()

## Create the nodes in the graph
Start with creating beneficiary nodes and attaching the chronic conditions they have as relationships (includes renal disease)

In [5]:
# Create the Condition nodes 
gds.run_cypher('create constraint if not exists for (n:Condition) require (n.id) is node key')
gds.run_cypher('''
    unwind [
        'RenalDiseaseIndicator',
        'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
        'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
        'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
        'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
        'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
        'ChronicCond_stroke'
    ] as conditionId
    merge (n:Condition{id: conditionId})
''')

In [ ]:
# Create Beneficiary nodes and connect them with HAS_CONDITION relationships to their corrosponding chronic conditions
gds.run_cypher('create constraint if not exists for (n:Beneficiary) require (n.id) is node key')
gds.run_cypher('''
    match (c:Condition)
    with collect(c) as conditions
    unwind $data as row
    merge (n:Beneficiary{id: row.BeneID})
        set n.dob = date(row.DOB),
            n.gender = row.Gender,
            n.race = row.Race,
            n.age = row.Age,
            n.state = row.State,
            n.county = row.County
    with conditions, n, row
    call {
        with row, conditions, n
        foreach(
            c in [x in conditions where row[x.id] = 1 or row[x.id] = 'Y' | x] |
            merge (n)-[:HAS_CONDITION]->(c)
        )
    }
''', params = {'data': data.to_dict('records')})

In [ ]:
# Add date of death (DOD) to beneficiaries who have passed away

dead = data[['BeneID','DOD']].dropna()
gds.run_cypher(''' 
    unwind $data as row
    match (n:Beneficiary{id: row.BeneID})
        set n.dod = date(row.DOD)
''', params = {'data': dead.to_dict('records')})

In [ ]:
# Create Provider nodes

gds.run_cypher('create constraint if not exists for (n:Provider) require (n.id) is node key')
label_dist = gds.run_cypher('''
    unwind $data as row
    merge (n:Provider{id: row.Provider})
        set n.fraud = case row.PotentialFraud when 1 then true else false end
    return n.fraud as is_fraud, count(*) as count
''', params = {'data': data.to_dict('records')})
label_dist.head()


### Create Claims Data

Going to create a node for each Claim, Provider, Physician, Diagnosis, and Procedure code. Create the relationships between beneficiary and their claims.

In [ ]:
# Create claims data

# Create constraints on our new node labels
gds.run_cypher('create constraint if not exists for (n:Claim) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Provider) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Physician) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Diagnosis) require (n.id) is node key')
gds.run_cypher('create constraint if not exists for (n:Procedure) require (n.id) is node key')

claims_cypher = '''
    unwind $data as row
    merge (c:Claim{id: row.ClaimID})
        set c.startDate = date(row.ClaimStartDt),
            c.endDate = date(row.ClaimEndDt),
            c.admissionDate = date(row.AdmissionDt),
            c.dischargeDate = date(row.DischargeDt),
            c.deductibleAmt = row.DeductibleAmtPaid,
            c.reimbursedAmt = row.InscClaimAmtReimbursed,
            c.daysAdmitted = row.DaysAdmitted,
            c.daysClaimLasted = row.DaysClaimLasted,
            c.claimEndAfterDischarged = row.ClaimEndAfterDischarged 
    merge (p:Provider{id: row.Provider})
    merge (c)-[:SUBMITTED_BY]->(p)                       
    merge (b:Beneficiary{id: row.BeneID})
    merge (c)-[:CREATED_FOR]->(b)                     
    '''

gds.run_cypher(claims_cypher, params = {'data': data.to_dict('records')})

In [ ]:
# Create Diagnoses code nodes and create the relationship ADMIT_DIAGNOSIS

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Diagnosis{id: row.ClmAdmitDiagnosisCode})
    merge (c)-[:HAS_ADMISSION_WITH]->(d)                    
''', params = {'data': data.to_dict('records')})

In [ ]:
# Create Diagnoses code nodes and create the relationship ADMIT_DIAGNOSIS

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Diagnosis{id: row.DiagnosisGroupCode})
    merge (c)-[:HAS_GROUP_CODE_OF]->(d)                        
''', params = {'data': data.to_dict('records')})

In [ ]:
# Create the relationships between claims and claim diagnoses codes

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_1})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                            
''', params = {'data': data.to_dict('records')})

claim_code_2 = data[['ClaimID','ClmDiagnosisCode_2']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_2})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                             
''', params = {'data': claim_code_2.to_dict('records')})

claim_code_3 = data[['ClaimID','ClmDiagnosisCode_3']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_3})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                    
''', params = {'data': claim_code_3.to_dict('records')})

claim_code_4 = data[['ClaimID','ClmDiagnosisCode_4']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_4})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                      
''', params = {'data': claim_code_4.to_dict('records')})

claim_code_5 = data[['ClaimID','ClmDiagnosisCode_5']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_5}) 
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                     
''', params = {'data': claim_code_5.to_dict('records')})

claim_code_6 = data[['ClaimID','ClmDiagnosisCode_6']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_6})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                         
''', params = {'data': claim_code_6.to_dict('records')})

claim_code_7 = data[['ClaimID','ClmDiagnosisCode_7']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_7})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                          
''', params = {'data': claim_code_7.to_dict('records')})

claim_code_8 = data[['ClaimID','ClmDiagnosisCode_8']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_8})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                           
''', params = {'data': claim_code_8.to_dict('records')})

claim_code_9 = data[['ClaimID','ClmDiagnosisCode_9']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_9})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                             
''', params = {'data': claim_code_9.to_dict('records')})

claim_code_10 = data[['ClaimID','ClmDiagnosisCode_10']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Diagnosis{id: row.ClmDiagnosisCode_10})
    merge (c)-[:HAS_DIAGNOSIS_CODE_OF]->(d)                                                
''', params = {'data': claim_code_10.to_dict('records')})

### Add Procedure nodes and connect them with the claim nodes

In [ ]:
# Create the relationships between claims and procedure codes

procedure_code_1 = data[['ClaimID','ClmProcedureCode_1']].dropna()

gds.run_cypher('''
    unwind $data as row
    match (c:Claim)
    where c.id = row.ClaimID
    merge (d:Procedure{id: row.ClmProcedureCode_1}) 
    merge (c)-[:HAS_PROCEDURE_CODE_OF]->(d)
''', params = {'data': procedure_code_1.to_dict('records')})

procedure_code_2 = data[['ClaimID','ClmProcedureCode_2']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Procedure{id: row.ClmProcedureCode_2})
    merge (c)-[:HAS_PROCEDURE_CODE_OF]->(d)
''', params = {'data': procedure_code_2.to_dict('records')})

procedure_code_3 = data[['ClaimID','ClmProcedureCode_3']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Procedure{id: row.ClmProcedureCode_3})
    merge (c)-[:HAS_PROCEDURE_CODE_OF]->(d)
''', params = {'data': procedure_code_3.to_dict('records')})

procedure_code_4 = data[['ClaimID','ClmProcedureCode_4']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Procedure{id: row.ClmProcedureCode_4})
    merge (c)-[:HAS_PROCEDURE_CODE_OF]->(d)
''', params = {'data': procedure_code_4.to_dict('records')})

procedure_code_5 = data[['ClaimID','ClmProcedureCode_5']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (c:Claim{id: row.ClaimID})
    merge (d:Procedure{id: row.ClmProcedureCode_5})
    merge (c)-[:HAS_PROCEDURE_CODE_OF]->(d)
''', params = {'data': procedure_code_5.to_dict('records')})

### Add the Physician nodes

In [ ]:
# Add Physicians and their realtionship to the claim

# Add attending physicians
attend = data[['ClaimID','AttendingPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.AttendingPhysician})
    merge (n)-[:HAS_ATTENDING]->(ap)                                                   
''', params = {'data': attend.to_dict('records')})

In [ ]:
# Add operating physicians
operating = data[['ClaimID','OperatingPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.OperatingPhysician})
    merge (n)-[:HAS_OPERATING]->(ap)                                               
''', params = {'data': operating.to_dict('records')})

In [ ]:
# Add other physicians
other = data[['ClaimID','OtherPhysician']].dropna()

gds.run_cypher(''' 
    unwind $data as row
    match (n:Claim{id: row.ClaimID})
    merge (ap:Physician{id: row.OtherPhysician})
    merge (n)-[:HAS_OTHER]->(ap)                                                    
''', params = {'data': other.to_dict('records')})

This marks the end of data loading here is the final schema of the data. To see this in the sandbox execute: CALL db.schema.visualization()